# Семинар 2: Многомерный анализ и выявление режимов в метеорологических данных (Jena Climate)

Этот ноутбук — логическое продолжение базового EDA. Здесь выполняется:
1) анализ избыточности и связей в многомерных данных,
2) линейное понижение размерности (PCA/EOF),
3) нелинейные проекции (t-SNE, UMAP),
4) кластеризация режимов (KMeans),
5) интерпретация результатов с точки зрения физики атмосферы.

**Датасет:** Jena Climate (2009–2016). Если CSV-файл не найден, будет создан небольшой синтетический набор с такой же структурой.


## Подготовка среды и загрузка данных

**Задание (к прочтению):**
- Указать путь к CSV-файлу Jena (если есть) или оставить по умолчанию.
- Выполнить ячейку и проверить: число строк/столбцов, временной охват.


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [ ]:
try:
    from sklearn.manifold import TSNE
    TSNE_AVAILABLE = True
except Exception:
    TSNE_AVAILABLE = False

In [ ]:
# UMAP (опционально)
try:
    import umap.umap_ as umap
    UMAP_AVAILABLE = True
except Exception:
    UMAP_AVAILABLE = False

In [ ]:
# Для показа таблиц (если поддерживается средой)
try:
    from caas_jupyter_tools import display_dataframe_to_user
except Exception:
    display_dataframe_to_user = None

In [ ]:
DATA_PATH = "/mnt/data/jena_climate_2009_2016.csv"  # при необходимости измените путь

In [ ]:
def load_or_mock_jena(path: str) -> pd.DataFrame:
    cols = [
        "Date Time","p (mbar)","T (degC)","Tpot (K)","Tdew (degC)","rh (%)",
        "VPmax (mbar)","VPact (mbar)","VPdef (mbar)","sh (g/kg)","H2OC (mmol/mol)",
        "rho (g/m**3)","wv (m/s)","max. wv (m/s)","wd (deg)"
    ]
    if os.path.exists(path):
        df = pd.read_csv(path)
        miss = [c for c in cols if c not in df.columns]
        if miss:
            raise ValueError(f"Отсутствуют ожидаемые столбцы: {miss}")
    df["Date Time"] = pd.to_datetime(df["Date Time"])
    return df.sort_values("Date Time").set_index("Date Time")

In [ ]:
df = load_or_mock_jena(DATA_PATH)

In [ ]:
# числовые признаки и часовые средние (ускоряет дальнейшие шаги)
num_df = df.select_dtypes(include=[np.number])
num_df_hr = num_df.resample("1H").mean().dropna(how="any")

In [ ]:
print("Размер исходных данных:", df.shape)
print("Размер часовых средних:", num_df_hr.shape)

## Этап 1. Избыточность и взаимосвязи в многомерных данных

**Задание.**  
1. Вычислить корреляционную матрицу по числовым переменным (часовые средние).  
2. Построить тепловую карту корреляций.  
3. Построить *scatter matrix* по подмножеству переменных (`T (degC)`, `rh (%)`, `p (mbar)`, `wv (m/s)`).  
4. Сделать краткие наблюдения о сильных/слабых связях (для обсуждения устно).


In [ ]:
# Корреляции
corr = num_df_hr.corr()

In [ ]:
plt.figure(figsize=(8, 6))
im = plt.imshow(corr.values, aspect="auto")
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.index)), corr.index)
plt.title("Корреляционная матрица (часовые средние)")
plt.tight_layout()
plt.show()

In [ ]:
# Scatter matrix (подмножество)
from pandas.plotting import scatter_matrix

In [ ]:
subset_cols = [c for c in ["T (degC)", "rh (%)", "p (mbar)", "wv (m/s)"] if c in num_df_hr.columns]

In [ ]:
if len(subset_cols) >= 2:
    smpl = num_df_hr[subset_cols].dropna().sample(min(2000, len(num_df_hr)), random_state=42)
    scatter_matrix(smpl, figsize=(7, 7), diagonal='hist')
    plt.suptitle("Scatter Matrix (подвыборка часовых средних)")
    plt.tight_layout()
    plt.show()
else:
    print("Недостаточно колонок для scatter matrix.")

## Этап 2. PCA / EOF-анализ (линейное понижение размерности)

**Задание.**  
1. Стандартизировать признаки.  
2. Выполнить PCA и вывести доли объяснённой дисперсии (scree-plot).  
3. Построить проекцию на первые две главные компоненты.  
4. Обсудить (устно), какие переменные доминируют в PC1/PC2 (через `pca.components_`).


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
# Стандартизация и PCA
scaler = StandardScaler()
X = scaler.fit_transform(num_df_hr.values)

In [ ]:
pca = PCA(n_components=6, random_state=42)
X_pca = pca.fit_transform(X)

In [ ]:
# Scree-plot
plt.figure(figsize=(6, 3.5))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1),
         pca.explained_variance_ratio_, marker='o')
plt.title("PCA: доля объяснённой дисперсии по компонентам")
plt.xlabel("Компонента")
plt.ylabel("Доля дисперсии")
plt.tight_layout()
plt.show()

In [ ]:
# Проекция PC1-PC2
plt.figure(figsize=(5, 4.5))
plt.scatter(X_pca[:, 0], X_pca[:, 1], s=8, alpha=0.5)
plt.title("PCA: проекция на 2 компоненты")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.tight_layout()
plt.show()

In [ ]:
# Вклады признаков в компоненты
loadings = pd.DataFrame(pca.components_.T, index=num_df_hr.columns, columns=[f"PC{i+1}" for i in range(pca.n_components_)])
print("Загрузки признаков (loadings) в компоненты:\n", loadings.head(10))

## Этап 3. Нелинейное понижение размерности (t-SNE, UMAP)

**Задание.**  
1. Выполнить t-SNE на подвыборке стандартизированных данных (для скорости).  
2. При наличии `umap-learn` — выполнить UMAP на подвыборке.  
3. Построить двумерные проекции; опционально — раскрасить точки по месяцу (`num_df_hr.index.month`).


In [ ]:
def try_tsne(X_scaled, max_points=1500, n_iter=500, perplexity=30, seed=42):
    # Защита от отсутствия TSNE
    try:
        from sklearn.manifold import TSNE
    except Exception:
        print("t-SNE недоступен (нет sklearn.manifold.TSNE). Шаг пропущен.")
        return None
    n = min(max_points, X_scaled.shape[0])
    Xs = X_scaled[:n, :]
    try:
        tsne = TSNE(n_components=2, init='pca',
                    perplexity=min(perplexity, max(5, n//10)), n_iter=n_iter, random_state=seed)
        emb = tsne.fit_transform(Xs)
        return emb
    except Exception as e:
        print(f"t-SNE не выполнен ({type(e).__name__}: {e}). Шаг пропущен.")
        return None

In [ ]:
# t-SNE
X_tsne = try_tsne(X, max_points=800, n_iter=350)
if X_tsne is not None:
    plt.figure(figsize=(5, 4.5))
    plt.scatter(X_tsne[:, 0], X_tsne[:, 1], s=8, alpha=0.6)
    plt.title("t-SNE (2D) на подвыборке")
    plt.xlabel("t-SNE 1")
    plt.ylabel("t-SNE 2")
    plt.tight_layout()
    plt.show()

In [ ]:
# UMAP (при наличии)
try:
    import umap.umap_ as umap
    UMAP_AVAILABLE = True
except Exception:
    UMAP_AVAILABLE = False

In [ ]:
if UMAP_AVAILABLE:
    try:
        reducer = umap.UMAP(n_neighbors=30, min_dist=0.1, n_components=2, random_state=42)
        n = min(4000, X.shape[0])
        X_umap = reducer.fit_transform(X[:n, :])
        plt.figure(figsize=(5, 4.5))
        plt.scatter(X_umap[:, 0], X_umap[:, 1], s=8, alpha=0.6)
        plt.title("UMAP (2D) на подвыборке")
        plt.xlabel("UMAP 1")
        plt.ylabel("UMAP 2")
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f"UMAP не выполнен ({type(e).__name__}: {e}). Шаг пропущен.")
else:
    print("UMAP не установлен. Шаг пропущен.")

## Этап 4. Кластеризация состояний (KMeans на PCA-пространстве)

**Задание.**  
1. Выполнить KMeans на первых 3 PC.  
2. Визуализировать кластеры в плоскости PC1–PC2.  
3. (Опционально) Попробовать разные `k` и обсудить изменение структуры.


In [ ]:
from sklearn.cluster import KMeans

In [ ]:
k = 5  # можно менять (3–6)
kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
labels = kmeans.fit_predict(X_pca[:, :3])

In [ ]:
plt.figure(figsize=(5.5, 5))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, s=8, alpha=0.6)
plt.title(f"KMeans на PC1–PC2 (k={k})")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.tight_layout()
plt.show()

## Этап 5. Интерпретация кластеров и физический смысл

**Задание.**  
1. Вернуть метки кластеров к исходным часовым данным.  
2. Посчитать средние профили переменных по кластерам.  
3. Построить z-нормированные «параллельные профили» для нескольких ключевых переменных.  
4. Сформулировать физическую интерпретацию кластеров (обсуждение устно).


In [ ]:
clustered = num_df_hr.copy()
clustered["cluster"] = labels
cluster_means = clustered.groupby("cluster").mean()

In [ ]:
print("\nСредние профили переменных по кластерам:\n", cluster_means)
if display_dataframe_to_user is not None:
    try:
        display_dataframe_to_user("Средние профили по кластерам", cluster_means)
    except Exception:
        pass

In [ ]:
plot_vars = [c for c in ["T (degC)", "rh (%)", "p (mbar)", "wv (m/s)", "VPdef (mbar)"] if c in cluster_means.columns]
if len(plot_vars) >= 3:
    cm = cluster_means[plot_vars]
    cm_norm = (cm - cm.mean()) / cm.std(ddof=0)
    x = np.arange(len(plot_vars))
    plt.figure(figsize=(7, 4))
    for idx in cm_norm.index:
        plt.plot(x, cm_norm.loc[idx].values, marker='o', label=f"cluster {idx}")
    plt.xticks(x, plot_vars)
    plt.title("Средние профили (z-нормированные) по кластерам")
    plt.xlabel("Переменные")
    plt.ylabel("Z-нормированные значения")
    plt.tight_layout()
    plt.show()
else:
    print("Недостаточно переменных для параллельных профилей.")